# Random Forest
Random Forest models are an ensemble learning method that tend to perform well because they build a series or ensemble of descision trees. This helps improve the accuracy by merging and leveraging the results of those trees. Random Forests are also great when trying to reduce overfitting.
For me the model resluted in a mediocre performance. It beat my baseline accuracy score but only marginally at 68%. I would have loved to tune this with model more via it's hyperparameters. But my computer lacks the processing power to run a more complex Gridsearch.

In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
import re
import pickle

# Loading data

In [3]:
combined = pd.read_pickle('../data/combined.pkl')


In [ ]:
X_train = pd.read_pickle('../data/X_train.pkl')
X_test = pd.read_pickle('../data/X_test.pkl')
y_train = pd.read_pickle('../data/y_train.pkl')
y_test = pd.read_pickle('../data/y_test.pkl')

# Setting up the pipeline

In [11]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('rfc',RandomForestClassifier(n_jobs=3,max_depth=53))
])
    

# Setting up the parameter grid

In [12]:
param_grid =  {
    'tfidf__min_df': np.arange(1,3,2),
    'tfidf__max_df': [0.9, 0.95],
    'rfc__n_estimators':[50,100],
    'rfc__min_samples_leaf': [1,2]
}

# Running a gridsearch

In [13]:
gs = GridSearchCV(pipe,param_grid=param_grid, verbose=1)

# Fitting the model

In [14]:
gs.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  8.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__min_df': array([1]), 'tfidf__max_df': [0.9, 0.95], 'rfc__n_estimators': [50, 100], 'rfc__min_samples_leaf': [1, 2]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

# Scoring the model
This data had decent performance. The amount of time it takes to fit was the only thing the kept me from tuning this model further.

In [15]:
gs.score(X_train,y_train)

0.6877538199196751

In [16]:
gs.score(X_test,y_test)

0.6610852740234017

In [ ]:
combined['subreddit'].value_counts()/combined.shape[0]

# Best estimators


In [17]:
gs.best_estimator_

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [26]:
gs.predict(['The jobs numbers were great, the numbers have been incredible.'])

array(['LateStageCapitalism'], dtype=object)

In [2]:
with open('../assets/random_forest.pkl','wb+') as f:
    pickle.dump(gs,f)